In [35]:
import base64
import requests
import cv2
import time
import json
from pathlib import Path
import openai
from IPython.display import display, Image, Audio
import os

# OpenAI API Key
api_key = "your api key"

# Function to convert a frame to base64
def encode_frame_to_base64(frame):
    _, buffer = cv2.imencode('.jpg', frame)
    return base64.b64encode(buffer).decode('utf-8')

# Function to read video and convert to base64
def get_base64_frames(video_path):
    video = cv2.VideoCapture(video_path)
    base64_frames = []
    while video.isOpened():
        success, frame = video.read()
        if not success:
            break
        base64_image = encode_frame_to_base64(frame)
        base64_frames.append(base64_image)
    video.release()
    return base64_frames

# Function to make an API call to OpenAI
def analyze_frame(base64_image, headers):
    payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "These are frames from a video that I want to upload. Generate a compelling description that I can upload along with the video."

                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.content

# The main function where the script starts
def main():
    video_path = "data/testshort.mp4"
    frames = get_base64_frames(video_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    all_contents = []  # List to store all content strings
    # Loop through all base64 frames and analyze them
    for i, base64_image in enumerate(frames):
        if i % 60 != 0:  # Skip if it's not the 10th frame
            continue
        result = analyze_frame(base64_image, headers)
        response_json = json.loads(result.decode('utf-8'))
        print(response_json)
        # Extract the content from the response
        content = response_json['choices'][0]['message']['content']
        all_contents.append(content)
        print(content)
        
        # To prevent hitting the API rate limits
        time.sleep(0.01)  # Adjust as per rate limits
         # After collecting all contents, write them to a file
        with open('analyzed_contents.txt', 'w', encoding='utf-8') as file:
            for content in all_contents:
                file.write(content + "\n\n")
    

    response = requests.post(
    "https://api.openai.com/v1/audio/speech",
    headers={
        "Authorization": f"Bearer {api_key}",
    },
    json={
        "model": "tts-1",
        "input": all_contents,
        "voice": "onyx",
    },
    )
    audio = b""
    for chunk in response.iter_content(chunk_size=1024 * 1024):
        audio += chunk
    Audio(audio)
    

if __name__ == "__main__":
    main()

{'error': {'message': 'Rate limit reached for gpt-4-vision-preview in organization org-aw32Cpn0G0waQQnETsRx0S7E on requests per day. Limit: 100 / day. Please try again in 14m24s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
键错误: 'choices'
